In [ ]:
"""face_trainer.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/17EdTwSvxibQlCuaADIKqSrAwoNS14uXE

# Face Recognition Model Trainer

This notebook trains a robust face recognition model that can identify people even when their images are distorted by various environmental conditions like blur, fog, low light, noise, rain, etc.

## Problem Statement

The goal is to create a face recognition system that:
- Works with a large dataset of people
- Handles multiple types of image distortions
- Achieves high accuracy (>90%) on validation data
- Uses efficient training techniques

## Technical Approach

- **Transfer Learning**: Uses ResNet50V2 pre-trained on ImageNet
- **Mixed Precision**: 2x faster training with 50% less memory usage
- **Advanced Augmentation**: Comprehensive data augmentation for robustness
- **Smart Callbacks**: Model checkpointing, early stopping, and learning rate reduction

## 1. Import Libraries and Setup
"""

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import cv2
from glob import glob
import pandas as pd
from collections import Counter

In [ ]:
print("✅ All libraries imported successfully!")

In [ ]:
"""## 2. Configuration Settings"""

In [ ]:
# Configuration for full dataset training
IMG_SIZE = 224
BATCH_SIZE = 32  # Increased batch size for efficiency
EPOCHS = 75
LEARNING_RATE = 5e-5  # Lower learning rate for fine-tuning
USE_MIXED_PRECISION = True  # Enable mixed precision for faster training

In [ ]:
print(f"📊 Configuration:")
print(f"   Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Mixed Precision: {USE_MIXED_PRECISION}")

In [ ]:
# Mixed precision setup
if USE_MIXED_PRECISION:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print("🚀 Mixed precision enabled for faster training")

In [ ]:
"""## 3. Dataset Loading Function"""

In [ ]:
def load_dataset():
    """
    Load complete dataset with all people and their distorted images

    Returns:
        tuple: (images, labels, distortion_types)
    """
    print("🔄 Loading complete dataset...")

    dataset_path = "dataset/Task_B/train"
    people_dirs = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])

    print(f"📁 Found {len(people_dirs)} people in dataset")

    images = []
    labels = []
    distortion_types = []

    # Track statistics
    total_images = 0
    skipped_people = 0

    for i, person_dir in enumerate(people_dirs):
        if i % 100 == 0:
            print(f"⏳ Processing person {i+1}/{len(people_dirs)}: {person_dir}")

        person_path = os.path.join(dataset_path, person_dir)

        # Load main image
        main_image_path = os.path.join(person_path, f"{person_dir}.jpg")
        if os.path.exists(main_image_path):
            try:
                img = cv2.imread(main_image_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    images.append(img)
                    labels.append(person_dir)
                    distortion_types.append('clean')
                    total_images += 1
            except Exception as e:
                print(f"❌ Error loading main image {main_image_path}: {e}")

        # Load distorted images
        distortion_path = os.path.join(person_path, "distortion")
        if os.path.exists(distortion_path):
            distortion_files = glob(os.path.join(distortion_path, "*.jpg")) + glob(os.path.join(distortion_path, "*.png"))

            for img_file in distortion_files:
                try:
                    img = cv2.imread(img_file)
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                        images.append(img)
                        labels.append(person_dir)
                        total_images += 1

                        # Determine distortion type
                        filename = os.path.basename(img_file)
                        if 'blurred' in filename:
                            distortion_types.append('blur')
                        elif 'foggy' in filename:
                            distortion_types.append('fog')
                        elif 'lowlight' in filename:
                            distortion_types.append('lowlight')
                        elif 'noisy' in filename:
                            distortion_types.append('noise')
                        elif 'rainy' in filename:
                            distortion_types.append('rain')
                        elif 'resized' in filename:
                            distortion_types.append('resized')
                        elif 'sunny' in filename:
                            distortion_types.append('overexposed')
                        else:
                            distortion_types.append('unknown')

                except Exception as e:
                    print(f"❌ Error loading distorted image {img_file}: {e}")
        else:
            skipped_people += 1

    print(f"\n📈 Dataset Statistics:")
    print(f"   Total people processed: {len(people_dirs)}")
    print(f"   People with distortion data: {len(people_dirs) - skipped_people}")
    print(f"   Total images loaded: {total_images}")
    print(f"   Unique classes: {len(set(labels))}")

    # Show distortion distribution
    distortion_counts = pd.Series(distortion_types).value_counts()
    print(f"\n🎭 Distortion distribution:")
    for dist_type, count in distortion_counts.items():
        print(f"   {dist_type}: {count} images")

    return np.array(images), np.array(labels), distortion_types

In [ ]:
"""## 4. Model Creation Function"""

In [ ]:
def create_model(num_classes):
    """
    Create enhanced model for full dataset training

    Args:
        num_classes (int): Number of unique people/classes

    Returns:
        tf.keras.Model: Compiled model ready for training
    """
    print(f"🏗️ Creating enhanced model for {num_classes} classes...")

    # Load pre-trained ResNet50V2
    base_model = ResNet50V2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # Freeze early layers, train later layers
    for layer in base_model.layers[:-30]:  # Freeze all except last 30 layers
        layer.trainable = False

    print(f"🔒 Frozen {len(base_model.layers[:-30])} layers, training {len(base_model.layers[-30:])} layers")

    # Create enhanced model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Output layer
    if USE_MIXED_PRECISION:
        # Use float32 for the final layer when using mixed precision
        x = tf.cast(x, tf.float32)
        predictions = Dense(num_classes, activation='softmax', dtype='float32')(x)
    else:
        predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile with optimized settings
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print("✅ Model created and compiled successfully!")
    return model

In [ ]:
"""## 5. Data Generator Function"""

In [ ]:
def create_data_generators(X_train, y_train, X_val, y_val):
    """
    Create advanced data generators with comprehensive augmentation

    Args:
        X_train, y_train: Training data and labels
        X_val, y_val: Validation data and labels

    Returns:
        tuple: (train_generator, val_generator)
    """
    print("🔄 Creating data generators with augmentation...")

    # Training data generator with advanced augmentation
    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest',
        rescale=1./255
    )

    # Validation data generator (minimal augmentation)
    val_datagen = ImageDataGenerator(
        rescale=1./255
    )

    # Create generators
    train_generator = train_datagen.flow(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    val_generator = val_datagen.flow(
        X_val, y_val,
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    print("✅ Data generators created successfully!")
    return train_generator, val_generator

In [ ]:
"""## 6. Training Function"""

In [ ]:
def train_model():
    """
    Train enhanced model on full dataset

    Returns:
        tuple: (trained_model, label_encoder, history, X, X_train, X_val, num_classes, distortion_types)
    """
    print("🚀 Starting full dataset robust training...")

    # Load complete dataset
    X, y, distortion_types = load_dataset()

    # Remove classes with only one sample
    label_counts = Counter(y)
    valid_classes = [label for label, count in label_counts.items() if count > 1]
    mask = np.isin(y, valid_classes)
    X = X[mask]
    y = y[mask]
    if isinstance(distortion_types, np.ndarray):
        distortion_types = distortion_types[mask]
    else:
        distortion_types = [d for d, m in zip(distortion_types, mask) if m]

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(label_encoder.classes_)

    print(f"\n📊 Dataset Info:")
    print(f"   Number of classes: {num_classes}")
    print(f"   Class distribution: min={np.bincount(y_encoded).min()}, max={np.bincount(y_encoded).max()}, avg={np.bincount(y_encoded).mean():.1f}")

    # Stratified split (guaranteed to work now)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
    )

    # Save validation set for evaluation
    np.save('X_val.npy', X_val)
    np.save('y_val.npy', y_val)

    print(f"📈 Data Split:")
    print(f"   Training samples: {len(X_train)}")
    print(f"   Validation samples: {len(X_val)}")

    # Create data generators
    train_generator, val_generator = create_data_generators(X_train, y_train, X_val, y_val)

    # Create enhanced model
    model = create_model(num_classes)

    # Print model summary
    print("\n📋 Model Summary:")
    model.summary()

    # Advanced callbacks
    callbacks = [
        ModelCheckpoint(
            'models/best_face_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=8,
            min_lr=1e-7,
            verbose=1
        ),
        CSVLogger(
            'results/training_log.csv',
            append=True
        )
    ]

    # Train model
    print("\n🎯 Starting training...")
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=len(val_generator),
        callbacks=callbacks,
        verbose=1
    )

    return model, label_encoder, history, X, X_train, X_val, num_classes, distortion_types

In [ ]:
"""## 7. Visualization Function"""

In [ ]:
def plot_training_history(history):
    """
    Plot training history with comprehensive visualizations

    Args:
        history: Training history from model.fit()
    """
    print("📊 Creating training visualizations...")

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title('Model Accuracy', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Model Loss', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 3, 3)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
    plt.title('Validation Accuracy Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.axhline(y=0.9, color='green', linestyle='--', label='90% Target', linewidth=2)
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('results/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✅ Training visualizations saved!")

In [ ]:
"""## 8. Results Saving Function"""

In [ ]:
def save_results(X, X_train, X_val, num_classes, history, distortion_types):
    """
    Save comprehensive training results

    Args:
        X: Complete dataset
        X_train, X_val: Training and validation splits
        num_classes: Number of classes
        history: Training history
        distortion_types: List of distortion types
    """
    print("💾 Saving comprehensive results...")

    with open('results/training_results.txt', 'w') as f:
        f.write(f"Face Recognition Training Results\n")
        f.write(f"==================================\n")
        f.write(f"Total images: {len(X)}\n")
        f.write(f"Number of people: {num_classes}\n")
        f.write(f"Training samples: {len(X_train)}\n")
        f.write(f"Validation samples: {len(X_val)}\n")
        f.write(f"Batch size: {BATCH_SIZE}\n")
        f.write(f"Learning rate: {LEARNING_RATE}\n")
        f.write(f"Mixed precision: {USE_MIXED_PRECISION}\n")
        f.write(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}\n")
        f.write(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}\n")
        f.write(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}\n")
        f.write(f"Training epochs: {len(history.history['accuracy'])}\n")

        # Distortion statistics
        f.write(f"\nDistortion Statistics:\n")
        distortion_df = pd.DataFrame({'distortion': distortion_types})
        distortion_counts = distortion_df['distortion'].value_counts()
        for dist_type, count in distortion_counts.items():
            f.write(f"  {dist_type}: {count} images\n")

    print("✅ Results saved to 'results/training_results.txt'")

In [ ]:
"""## 9. Main Execution"""

In [ ]:
# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

In [ ]:
print("📁 Directories created successfully!")

In [ ]:
# Train model
model, label_encoder, history, X, X_train, X_val, num_classes, distortion_types = train_model()

In [ ]:
import pickle

In [ ]:
with open('models/enhanced_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("✅ Label encoder saved to 'models/enhanced_label_encoder.pkl'")

In [ ]:
# Plot training history
plot_training_history(history)

In [ ]:
# Save results
save_results(X, X_train, X_val, num_classes, history, distortion_types)

In [ ]:
# Final evaluation
print(f"\n🎉 Training completed!")
print(f"🏆 Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"📊 Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
if max(history.history['val_accuracy']) >= 0.9:
    print("🎯 SUCCESS: Model achieved 90%+ accuracy on full dataset!")
else:
    print(f"⚠️ Model achieved {max(history.history['val_accuracy']):.1%} accuracy (target: 90%)")

In [ ]:
print("\n" + "="*80)
print("🎯 TRAINING COMPLETED SUCCESSFULLY!")
print("="*80)
print("📁 Files created:")
print("  • models/best_face_model.h5 - Trained model")
print("  • results/training_history.png - Training plots")
print("  • results/training_log.csv - Training history")
print("  • results/training_results.txt - Detailed results")
print("\n🚀 This model is ready for face recognition tasks!")
print("="*80)